# Data Anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage30yr""","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage30yr""","""Unknown""",669,"""Laurianneshire""","""AutoLoans""","""Accepted"""
"""MoneyMarketSavingsAccount""","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmieshire""","""DepositAccounts""","""Accepted"""
"""UPlusFinPersonal""","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccounts""","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,filename,Context_Name,IH_PREDICTOR_0,Decision_Outcome
str,f64,str,str,str,str,bool
"""5565018800276297245""",1.3185e19,"""5856826697151246967""","""../../../../data/SampleHDS.jso…","""3165828713594416045""","""5763778890622320731""",false
"""4220923862891400722""",1.8017e19,"""3465216081957903020""","""../../../../data/SampleHDS.jso…","""3165828713594416045""","""17854784191108743952""",true
"""12205350029517737996""",3.4794e18,"""11023970938746469468""","""../../../../data/SampleHDS.jso…","""17360784779795211625""","""5763778890622320731""",false
"""14060860551100596309""",1.9906e18,"""3465216081957903020""","""../../../../data/SampleHDS.jso…","""2502890027456889916""","""5763778890622320731""",false
"""6824095567266657391""",8.1381e18,"""5856826697151246967""","""../../../../data/SampleHDS.jso…","""2502890027456889916""","""17587893398981663078""",true
"""2355359715695589929""",1.4398e19,"""11023970938746469468""","""../../../../data/SampleHDS.jso…","""17949994353248198069""",null,false
"""9743415053111744659""",1.5062e19,"""11023970938746469468""","""../../../../data/SampleHDS.jso…","""2502890027456889916""","""17587893398981663078""",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_City': 'PREDICTOR_0',
 'Customer_CLV': 'PREDICTOR_1',
 'Customer_MaritalStatus': 'PREDICTOR_2',
 'filename': 'filename',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,filename,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str,str
"""4004549653168640002""",1.5818e19,"""15029343331266295276""","""../../../../data/SampleHDS.jso…","""FirstMortgage30yr""","""12775381765945713764""","""Rejected"""
"""1833447162809356376""",1.6596e19,"""6353536280384581992""","""../../../../data/SampleHDS.jso…","""FirstMortgage30yr""","""9453409868886306982""","""Accepted"""
"""6044845130636484325""",6.6227e18,"""4984329304133979504""","""../../../../data/SampleHDS.jso…","""MoneyMarketSavingsAccount""","""12775381765945713764""","""Rejected"""
"""2318318381508552261""",1.3091e18,"""6353536280384581992""","""../../../../data/SampleHDS.jso…","""BasicChecking""","""12775381765945713764""","""Rejected"""
"""10414374308868003429""",5.6559e18,"""15029343331266295276""","""../../../../data/SampleHDS.jso…","""BasicChecking""","""4862555200493705739""","""Accepted"""
"""1765112207319902096""",1.5368e19,"""4984329304133979504""","""../../../../data/SampleHDS.jso…","""UPlusFinPersonal""",null,"""Rejected"""
"""3029520815025042844""",2.0143e18,"""4984329304133979504""","""../../../../data/SampleHDS.jso…","""BasicChecking""","""4862555200493705739""","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,filename,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str,str
"""7645377549233683863""",3.8498e18,"""1777096350238255819""","""../../../../data/SampleHDS.jso…","""FirstMortgage30yr""","""2785132465348534201""","""Rejected"""
"""7474026283469500281""",1.4536e19,"""3798541720252534146""","""../../../../data/SampleHDS.jso…","""FirstMortgage30yr""","""9197922674177966128""","""Accepted"""
"""13227929241415219705""",1.0046e19,"""9934115004748427206""","""../../../../data/SampleHDS.jso…","""MoneyMarketSavingsAccount""","""2785132465348534201""","""Rejected"""
"""18037667313826183204""",1.2297e19,"""3798541720252534146""","""../../../../data/SampleHDS.jso…","""BasicChecking""","""2785132465348534201""","""Rejected"""
"""12692780900903441771""",8.9171e18,"""1777096350238255819""","""../../../../data/SampleHDS.jso…","""BasicChecking""","""3805229203175271739""","""Accepted"""
"""4389908551137424160""",1.7892e19,"""9934115004748427206""","""../../../../data/SampleHDS.jso…","""UPlusFinPersonal""",null,"""Rejected"""
"""4023864974873284358""",2.4227e18,"""9934115004748427206""","""../../../../data/SampleHDS.jso…","""BasicChecking""","""3805229203175271739""","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_City=PREDICTOR_0
Customer_CLV=PREDICTOR_1
Customer_MaritalStatus=PREDICTOR_2
filename=filename
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str
"""1796799448019375302""",4.1316e18,"""16182508356800230504""","""FirstMortgage30yr""","""7119676613440967820""","""Rejected"""
"""520310407117878958""",5.1759e18,"""4003943536054469557""","""FirstMortgage30yr""","""15485573365642416268""","""Accepted"""
"""13939711651552270405""",8.0650e18,"""15908238847607988474""","""MoneyMarketSavingsAccount""","""7119676613440967820""","""Rejected"""
"""4918292960533066692""",2.1056e18,"""4003943536054469557""","""BasicChecking""","""7119676613440967820""","""Rejected"""
"""15698310252231812593""",7.1883e18,"""16182508356800230504""","""BasicChecking""","""11035385418683566402""","""Accepted"""
"""16925197139920173102""",4.1871e18,"""15908238847607988474""","""UPlusFinPersonal""",null,"""Rejected"""
"""8858378051492226294""",7.1199e18,"""15908238847607988474""","""BasicChecking""","""11035385418683566402""","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.



ComputeError: AttributeError: 'int' object has no attribute 'encode'